In [1]:
import pandas as pd

import protfasta
import requests
import re
import warnings
warnings.filterwarnings("ignore")
import glob
import collections


from Bio import AlignIO
import matplotlib.pyplot as plt
from Bio.Align import AlignInfo

from Bio.Align import MultipleSeqAlignment
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

import rich
from rich_msa import RichAlignment

import logomaker

#Import modules
import gzip, logomaker, matplotlib.pyplot as plt, numpy as np, os, pandas as pd, protfasta


import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import itertools


from scipy import stats

Matplotlib is building the font cache; this may take a moment.


# Percent identity
Saving the percent identity of each position, relative to the lambert TFs sequence.

Step 0: Check how many of CC's genes already have the correct isoform.

In [34]:
# Loading in info about the ADs
cc_names = pd.read_excel("../data/gene_names_with_location_and_pmvs.xlsx")
cc_names = cc_names.rename(columns = {"gene" : "Gene", "start" : "Start", "end" : "End"})
known_ADs = pd.read_csv("../output/known_ADs_considering_isoforms_and_canonical.csv")
known_ADs = known_ADs[known_ADs["TileType"] == "TF"]
cc_names = pd.merge(known_ADs[["uniprotID", "Gene", "Start", "End"]], cc_names, how = "left")
# ensts = pd.read_csv("../data/SFARI_TFs_with_ENST_corrected.csv", index_col = 0)
# cc_names = pd.merge(cc_names, ensts)
# cc_names["ENST"] = cc_names["ENST"].str.split(".").str[0]
# cc_names["len"] = cc_names["End"] - cc_names["Start"] + 1
# cc_names = cc_names.rename(columns = {"Start" : "start", "End" : "end"}).drop(columns = "pMVS #")
# # cc_names.at[9, "end"] = 93 # to make consistent with other table
# cc_names["Gene Name"] = cc_names["Gene Name"].fillna("AD")
cc_TFs = cc_names[["uniprotID"]].drop_duplicates()

gene_names = pd.read_csv("../data/TF_uniprotIDs_to_gene_names.tsv", sep = "\t")
gene_names["uniprotID"] = gene_names["From"]
gene_names["Gene"] = gene_names["Gene Names"].str.split(" ").str[0]
cc_TFs = pd.merge(cc_TFs, gene_names[["uniprotID", "Gene"]])
cc_TFs

,uniprotID,Gene
0,Q8IUX7,AEBP1
1,Q8WYP5,AHCTF1
2,P35869,AHR
3,O43918,AIRE
4,O43823,AKAP8
...,...,...
441,Q8N1G0,ZNF687
442,P17040,ZSCAN20
443,P98168,ZXDA
444,P98169,ZXDB


In [35]:
# Assuming everything in prot alignments folder is actually correct
correc_isoforms = glob.glob("../data/zoonomia_toga_mca/prot_alignments/*")
correc_isoform_genes = [_.split("/")[-1].split(".")[0] for _ in correc_isoforms]
for gene in cc_TFs["Gene"]:
    if gene not in correc_isoform_genes:
        print(gene)

AHCTF1
BMAL1
ATF6
BMAL2
CREB1
CXXC1
DDIT3
DUX4
EBF3
EGR2
ELF5
EOMES
ERG
ESRRA
ESRRG
ETS1
ETS2
ETV1
ETV2
FOXA2
FOXM1
GATA3
GATA4
GLIS1
GLIS3
HIF1A
HIVEP3
HMGA2
IRF3
IRF5
IRF7
IRX4
KDM5B
KMT2A
LHX3
MAF
MEF2A
MEF2C
MEIS1
MIXL1
MYB
MYCL
MYT1L
NANOGP8
NFAT5
NFATC4
NFIA
NFIC
NFIX
NFKB1
NPAS3
NR1H3
NR1I2
NR1I3
NR3C1
NR4A1
NR4A3
NRF1
OTX2
PAX3
PAX6
PAX7
PAX8
PHF19
POU1F1
PITX2
PLSCR1
POU2F1
POU2F2
PPARA
SPI1
RARA
RORA
RUNX1
RUNX2
RUNX3
RXRB
SP100
SREBF1
STAT1
STAT3
STAT5A
STAT6
TBX4
TBXT
TCF12
TCF20
TCF3
TCF4
TEAD2
TET2
TFEB
THRB
TRERF1
USF2
VDR
ZBTB17
ZFX
ZFY
ZNF561
ZNF280D
ZNF426
ZXDA
ZXDB


Plan: For ERG and PAX6, align the correct isoform's sequence to the current protein alignment. Use that to save the percent identity per position.

# Add fragment: Align Lambert TFs sequence to the zoonomia alignment

In [36]:
lambert_TFs = pd.read_csv("../output/lambert_TFs_10-21-24_with_DBD_coords.csv", index_col = 0)
lambert_TFs

,id,ProteinSeq,DBD_coords_merged
0,sp|A0A087WUV0|ZN892_HUMAN Zinc finger protein ...,MEPEGRGSLFEDSDLLHAGNPKENDVTAVLLTPGSQELMIRDMAEA...,"[[221, 243], [249, 271], [277, 299], [305, 327..."
1,sp|A0AVK6|E2F8_HUMAN Transcription factor E2F8...,MENEKENLFCEPHKRGLMKTPLKESTTANIVLAEIQPDFGPLTTPT...,"[[114, 182], [262, 347]]"
2,sp|A0PJY2|FEZF1_HUMAN Fez family zinc finger p...,MDSSCHNATTKMLATAPARGNMMSTSKPLAFSIERIMARTPEPKAL...,"[[260, 282], [288, 310], [316, 338], [344, 366..."
3,sp|A1A519|F170A_HUMAN Protein FAM170A OS=Homo ...,MKRRQKRKHLENEESQETAEKGGGMSKSQEDALQPGSTRVAKGWSQ...,"[[1, 330]]"
4,sp|A1YPR0|ZBT7C_HUMAN Zinc finger and BTB doma...,MANDIDELIGIPFPNHSSEVLCSLNEQRHDGLLCDVLLVVQEQEYR...,"[[364, 386], [392, 414], [420, 442], [448, 469]]"
...,...,...,...
1608,sp|Q9Y6Q9|NCOA3_HUMAN Nuclear receptor coactiv...,MSGLGENLDPLASDSRKRKLPCDTPGQGLTCSGEKRRREQESKYIE...,"[[31, 83]]"
1609,sp|Q9Y6R6|Z780B_HUMAN Zinc finger protein 780B...,MVHGSVTFRDVAIDFSQEEWECLQPDQRTLYRDVMLENYSHLISLG...,"[[165, 187], [193, 215], [221, 243], [249, 271..."
1610,sp|Q9Y6X0|SETBP_HUMAN SET-binding protein OS=H...,MESRETLSSSRQRGGESDFLPVSSAKPPAAPGCAGEPLLSTPGPGK...,"[[583, 596], [1015, 1027], [1450, 1462]]"
1611,sp|Q9Y6X8|ZHX2_HUMAN Zinc fingers and homeobox...,MASKRKSTTPCMVRTSQVVEQDVPEEVDRAKEKGIGTPQPDVAKDS...,"[[78, 101], [110, 133], [271, 317], [442, 496]..."


In [37]:
# ! mkdir ../data/zoonomia_toga_mca/prot_alignments_for_vis/
#! mkdir ../data/zoonomia_toga_mca/prot_seqs_for_vis/

In [38]:
full_prot_alignments = glob.glob("../data/zoonomia_toga_mca/prot_alignments/*")
full_prot_alignments[:2]

['../data/zoonomia_toga_mca/prot_alignments/TP63.fasta',
 '../data/zoonomia_toga_mca/prot_alignments/RARG.fasta']

In [39]:
for gene in cc_TFs["Gene"]:
    uniprotID = dict(zip(cc_TFs["Gene"], cc_TFs["uniprotID"]))[gene]
    df = lambert_TFs[lambert_TFs["id"].str.contains(uniprotID)]
    protfasta.write_fasta([("LambertSeq", df["ProteinSeq"].iloc[0])], "../data/zoonomia_toga_mca/prot_seqs_for_vis/" + gene)
    #display(df)

    if gene in correc_isoform_genes:
        command = "mafft --auto --quiet --add ../data/zoonomia_toga_mca/prot_seqs_for_vis/" + gene + " --reorder --thread -1 ../data/zoonomia_toga_mca/prot_alignments/" + gene + ".fasta > ../data/zoonomia_toga_mca/prot_alignments_for_vis/" + gene + ".fasta"
    else:
        command = "mafft --auto --quiet --add ../data/zoonomia_toga_mca/prot_seqs_for_vis/" + gene + " --reorder --thread -1 ../data/zoonomia_toga_mca/prot_alignments_wrong_isoforms/" + gene + ".fasta > ../data/zoonomia_toga_mca/prot_alignments_for_vis/" + gene + ".fasta"
     
    os.system(command)

/usr/local/bin/mafft: Cannot open ../data/zoonomia_toga_mca/prot_alignments_wrong_isoforms/BMAL1.fasta.

/usr/local/bin/mafft: line 1275: /var/folders/hw/xx051vr9457c7lrngf2mypgr0000gn/T/mafft.XXXXXXXXXX.HPI2KagoW7/_addfile: No such file or directory
awk: can't open file /var/folders/hw/xx051vr9457c7lrngf2mypgr0000gn/T/mafft.XXXXXXXXXX.HPI2KagoW7/addsize
 source line number 1
/usr/local/bin/mafft: line 1277: [: -eq: unary operator expected
/usr/local/bin/mafft: line 1291: /var/folders/hw/xx051vr9457c7lrngf2mypgr0000gn/T/mafft.XXXXXXXXXX.HPI2KagoW7/infile: No such file or directory
awk: can't open file /var/folders/hw/xx051vr9457c7lrngf2mypgr0000gn/T/mafft.XXXXXXXXXX.HPI2KagoW7/size
 source line number 1
awk: can't open file /var/folders/hw/xx051vr9457c7lrngf2mypgr0000gn/T/mafft.XXXXXXXXXX.HPI2KagoW7/size
 source line number 1
/usr/local/bin/mafft: line 1295: [: too many arguments
/usr/local/bin/mafft: line 1300: [: too many arguments
/usr/local/bin/mafft: line 1305: [: too many argumen

# Save percent identities for each

In [43]:
aligned_with_lambert = glob.glob("../data/zoonomia_toga_mca/prot_alignments_for_vis/*.fasta")
aligned_with_lambert = [_.split("/")[-1].split(".")[0] for _ in aligned_with_lambert]
aligned_with_lambert

['ZBTB17',
 'TP63',
 'RARG',
 'NCOA2',
 'ZNF148',
 'SPIB',
 'NEUROD6',
 'FOXN1',
 'HOXA6',
 'GRHL1',
 'YY2',
 'SRF',
 'ARID5A',
 'SOX2',
 'PAX6',
 'AR',
 'CREBZF',
 'ERG',
 'ZNF641',
 'OTX2',
 'ETV4',
 'SMAD1',
 'TRAFD1',
 'MYF6',
 'NR1I2',
 'SMAD3',
 'GCM1',
 'KDM5B',
 'E2F2',
 'MITF',
 'FOS',
 'MAFA',
 'SREBF2',
 'KLF1',
 'GRHL3',
 'MYOG',
 'VAX1',
 'ZSCAN20',
 'MEIS1',
 'THRB',
 'HBP1',
 'SOX18',
 'HOXB9',
 'AHR',
 'TIGD1',
 'LEUTX',
 'MSX2',
 'KLF16',
 'MLXIP',
 'KLF5',
 'NKX6-1',
 'FOXF2',
 'POU5F1',
 'FOXJ2',
 'RARA',
 'FOXR2',
 'MZF1',
 'CREM',
 'ETV2',
 'USF2',
 'ZNF687',
 'ELK4',
 'STAT3',
 'TWIST1',
 'EOMES',
 'MYBL1',
 'SOX4',
 'PAX2',
 'STAT1',
 'PRRX1',
 'TEF',
 'POU2AF1',
 'BSX',
 'TFEB',
 'SMAD5',
 'E2F4',
 'TIGD7',
 'NEUROD2',
 'SP9',
 'TBX21',
 'PHF1',
 'ZNF16',
 'PITX2',
 'KLF7',
 'HOXA2',
 'STAT6',
 'AHCTF1',
 'PAX5',
 'ZBTB48',
 'ELK1',
 'SOX1',
 'ARNT',
 'DMRT1',
 'ALX3',
 'MYF5',
 'ATMIN',
 'FOSL1',
 'HOXD9',
 'IKZF1',
 'NR1I3',
 'REL',
 'JUND',
 'OTX1',
 'E2F3',


In [44]:
for gene in cc_TFs["Gene"]:
    print(gene)
    if gene in aligned_with_lambert:
        alignment = AlignIO.read("../data/zoonomia_toga_mca/prot_alignments_for_vis/" + gene + ".fasta", "fasta")
        percent_identities = []
        summary_align = AlignInfo.SummaryInfo(alignment)
        
        for record in alignment:
            if "REFERENCE" in record.id:
                reference_seq = record.seq
    
            if "LambertSeq" in record.id:
                lambert_TF_seq = record.seq
        
        percent_identities = []
        positions = []
        
        # Iterate over each position in the alignment where ref_AA is not a gap
        for i in range(alignment.get_alignment_length()):
            ref_AA = reference_seq[i]
            lam_AA = lambert_TF_seq[i]
    
            
            # if ref_AA == "X" or ref_AA == "-":
            #     print(gene)
    
            if lam_AA != "-":
                # Get the column (position) for all sequences
                column = alignment[:, i]
            
                # Calculating percent identity
                count = column.count(ref_AA)
                percent_identity = 100 * count / len(column)
            
                # AAs.append(ref_AA)
                percent_identities.append(percent_identity)
                positions.append(lam_AA)
            
        percent_identities_df = pd.DataFrame({"percent_identity" : percent_identities, "pos" : positions})
        percent_identities_df.to_csv("../data/zoonomia_toga_mca/prot_alignment_percent_identities_for_vis/" + gene)
        #display(percent_identities_df)


AEBP1
AHCTF1
AHR
AIRE
AKAP8
AKAP8L
ALX1
ALX3
AR
ARGFX
ARID5A
ARNT
ARNT2
BMAL1
ARX
ATF2
ATF3
ATF4
ATF5
ATF6B
ATF6
ATF7
ATMIN
BMAL2
BSX
CC2D1A
CAMTA1
CAMTA2
CASZ1
CDX1
CDX2
CDX4
CEBPA
CEBPB
CEBPD
CEBPE
CREB3L1
CREB1
CREB3
CREB3L2
CREB3L3
CREB3L4
CREM
CRX
CSRNP1
CXXC1
DBP
DDIT3
DLX3
DMRT1
DMTF1
DUX4
E2F1
E2F2
E2F3
E2F4
E2F5
EBF1
EBF3
EGR1
EGR2
EGR3
EGR4
ELF1
ELF2
ELF3
ELF4
ELF5
ELK1
ELK3
ELK4
EOMES
EPAS1
ERG
ESR1
ESR2
ESRRA
ESRRG
ETS1
ETS2
ETV1
ETV2
ETV3
ETV4
ETV5
FLI1
ZFPM2
FOS
FOSB
FOSL1
FOXA1
FOXA2
FOXC1
FOXC2
FOXF1
FOXF2
FOXI1
FOXI3
FOXJ1
FOXJ2
FOXJ3
FOXM1
FOXN1
FOXN2
FOXN4
FOXO1
FOXO3
FOXO4
FOXO6
FOXR1
FOXR2
GATA1
GATA2
GATA3
GATA4
GATA5
GATA6
GCM1
GCM2
GLI1
GLI2
GLI3
GLIS1
GLIS2
GLIS3
GLMP
GMEB1
GMEB2
GRHL1
GRHL2
GRHL3
HAND2
HBP1
HHEX
HIC2
HIF1A
HIF3A
HINFP
HIVEP3
HLF
HMGA1
HMGA2
HNF1B
HNF4A
HOXA10
HOXA13
HOXA2
HOXA6
HOXA7
HOXB1
HOXB13
HOXB2
HOXB3
HOXB6
HOXB7
HOXB9
HOXD13
HOXD4
HOXD9
HSF2
HSF1
HSF5
IKZF1
IRF1
IRF2
IRF3
IRF5
IRF6
IRF7
IRX4
JAZF1
JRKL
JUN
JUNB
JUND
KAT7
KDM5B
KLF1
KLF

## A. Correct isoforms